In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import HTML, display
import tabulate
import re
import seaborn as sns
sns.set_theme(style="whitegrid")

In [2]:
api_data = pd.read_csv("../../data/api_data.csv")

print(f"CSV has {len(api_data)} projects.")

CSV has 2731 projects.


In [3]:
api_data

,id,project,project_new_ownername,chunks,repo_not_found,project_actual_ownername,resourcePath,stargazerCount,isFork,deleteBranchOnMerge,...,issues,pullRequests,releases,watchers,forks,forkCount,commits,contributors,branches,tags
0,3518171,sebastianbenz/Jnario,NaN,10372,False,sebastianbenz/Jnario,/sebastianbenz/Jnario,133.0,False,False,...,146.0,35.0,0.0,19.0,32.0,37.0,1303.0,9.0,19.0,20.0
1,3661343,CCI-MIT/XCoLab,NaN,6889,False,CCI-MIT/XCoLab,/CCI-MIT/XCoLab,15.0,False,False,...,9.0,288.0,0.0,8.0,10.0,10.0,10045.0,25.0,97.0,0.0
2,2524488,apache/accumulo,NaN,5141,False,apache/accumulo,/apache/accumulo,864.0,False,False,...,628.0,1394.0,0.0,88.0,333.0,351.0,10876.0,131.0,3.0,38.0
3,65885,android/platform_frameworks_base,aosp-mirror/platform_frameworks_base,4446,False,aosp-mirror/platform_frameworks_base,/aosp-mirror/platform_frameworks_base,9532.0,False,False,...,0.0,111.0,0.0,911.0,5360.0,5901.0,566598.0,763.0,250.0,1034.0
4,1006053,TeamDev-Ltd/OpenFaces,TeamDev-Archive/OpenFaces,3723,False,TeamDev-Archive/OpenFaces,/TeamDev-Archive/OpenFaces,28.0,False,False,...,173.0,3.0,0.0,19.0,14.0,15.0,3293.0,18.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2726,3467722,pingpongboss/Phone-Modem,NaN,1,False,pingpongboss/Phone-Modem,/pingpongboss/Phone-Modem,30.0,False,False,...,0.0,0.0,0.0,5.0,9.0,9.0,24.0,2.0,1.0,0.0
2727,3468682,htdvisser/Skt,NaN,1,True,htdvisser/Skt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2728,2130242,justjoheinz/STLDroid,NaN,1,False,justjoheinz/STLDroid,/justjoheinz/STLDroid,10.0,False,False,...,1.0,0.0,0.0,1.0,4.0,4.0,24.0,1.0,1.0,5.0
2729,3355503,SpoutDev/BukkitBridge,NaN,1,False,SpoutDev/BukkitBridge,/SpoutDev/BukkitBridge,19.0,False,False,...,0.0,53.0,0.0,17.0,9.0,9.0,248.0,21.0,1.0,0.0


In [4]:
conflicting_merges_per_project = pd.read_csv("../../data/number_conflicting_merges_project_selected.csv")
conflicting_merges_per_project

,id,project,nr_conflicting_merges
0,65885,android/platform_frameworks_base,3355
1,185026,freenet/fred,237
2,206437,apache/directory-server,18
3,217482,Ramblurr/Anki-Android,155
4,223355,alexo/wro4j,395
5,507775,elastic/elasticsearch,235
6,726492,getrailo/railo,163
7,762119,atlasapi/atlas,418
8,961036,hibernate/hibernate-orm,27
9,1006053,TeamDev-Ltd/OpenFaces,178


In [5]:
merges_per_project = pd.read_csv("../../data/number_merges_project_selected.csv")
merges_per_project

,id,project,nr_merges
0,65885,android/platform_frameworks_base,156732
1,185026,freenet/fred,1076
2,206437,apache/directory-server,50
3,217482,Ramblurr/Anki-Android,952
4,223355,alexo/wro4j,1458
5,507775,elastic/elasticsearch,2442
6,726492,getrailo/railo,671
7,762119,atlasapi/atlas,2284
8,961036,hibernate/hibernate-orm,152
9,1006053,TeamDev-Ltd/OpenFaces,611


In [6]:
merges_csv = pd.merge(merges_per_project, conflicting_merges_per_project, on='project', how='inner')
merges_csv = merges_csv.drop(columns=['id_y'])
merges_csv = merges_csv.rename(columns={'id_x':'id'})
merges_csv

,id,project,nr_merges,nr_conflicting_merges
0,65885,android/platform_frameworks_base,156732,3355
1,185026,freenet/fred,1076,237
2,206437,apache/directory-server,50,18
3,217482,Ramblurr/Anki-Android,952,155
4,223355,alexo/wro4j,1458,395
5,507775,elastic/elasticsearch,2442,235
6,726492,getrailo/railo,671,163
7,762119,atlasapi/atlas,2284,418
8,961036,hibernate/hibernate-orm,152,27
9,1006053,TeamDev-Ltd/OpenFaces,611,178


In [7]:
selected_dataset_2 = pd.read_csv("../../data/selected_dataset_2.csv")
selected_dataset_2

,chunk_id,developerdecision,line_start,line_end,line_separator,kind_conflict,url,project,project_user,project_name,...,Interface signature,Attribute,Interface declaration,Method signature,Class declaration,Static initializer,Assert statement,Return statement,Array access,Throw statement
0,777206,ConcatenationV2V1,427,436,434,"Attribute, Comment, Method invocation",https://github.com/Ramblurr/Anki-Android,Ramblurr/Anki-Android,Ramblurr,Anki-Android,...,0,1,0,0,0,0,0,0,0,0
1,777207,Version 1,2826,2871,2827,"Comment, Method declaration",https://github.com/Ramblurr/Anki-Android,Ramblurr/Anki-Android,Ramblurr,Anki-Android,...,0,0,0,0,0,0,0,0,0,0
2,777216,Combination,2458,2465,2463,"Comment, Method invocation, Variable",https://github.com/Ramblurr/Anki-Android,Ramblurr/Anki-Android,Ramblurr,Anki-Android,...,0,0,0,0,0,0,0,0,0,0
3,777218,Combination,2476,2482,2479,"Comment, Method invocation, Variable",https://github.com/Ramblurr/Anki-Android,Ramblurr/Anki-Android,Ramblurr,Anki-Android,...,0,0,0,0,0,0,0,0,0,0
4,777221,Version 1,750,753,751,"Method invocation, Variable",https://github.com/Ramblurr/Anki-Android,Ramblurr/Anki-Android,Ramblurr,Anki-Android,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57209,1357284,Version 1,3,7,5,Package declaration,https://github.com/sebastianbenz/Jnario,sebastianbenz/Jnario,sebastianbenz,Jnario,...,0,0,0,0,0,0,0,0,0,0
57210,1357285,Version 1,17,23,21,Attribute,https://github.com/sebastianbenz/Jnario,sebastianbenz/Jnario,sebastianbenz,Jnario,...,0,1,0,0,0,0,0,0,0,0
57211,1357286,Version 1,29,68,49,Method declaration,https://github.com/sebastianbenz/Jnario,sebastianbenz/Jnario,sebastianbenz,Jnario,...,0,0,0,0,0,0,0,0,0,0
57212,1357287,Version 1,3,7,5,Package declaration,https://github.com/sebastianbenz/Jnario,sebastianbenz/Jnario,sebastianbenz,Jnario,...,0,0,0,0,0,0,0,0,0,0


In [8]:
selected_dataset_2.columns

Index(['chunk_id', 'developerdecision', 'line_start', 'line_end',
       'line_separator', 'kind_conflict', 'url', 'project', 'project_user',
       'project_name', 'path', 'file_name', 'sha', 'leftsha', 'rightsha',
       'basesha', 'left_lines_added', 'left_lines_removed',
       'right_lines_added', 'right_lines_removed', 'conclusion_delay',
       'keyword_fix', 'keyword_bug', 'keyword_feature', 'keyword_improve',
       'keyword_document', 'keyword_refactor', 'keyword_update', 'keyword_add',
       'keyword_remove', 'keyword_use', 'keyword_delete', 'keyword_change',
       'leftCC', 'rightCC', 'fileCC', 'chunkAbsSize', 'chunkRelSize',
       'chunkPosition', 'fileSize', 'chunk_left_abs_size',
       'chunk_left_rel_size', 'chunk_right_abs_size', 'chunk_right_rel_size',
       'Branching time', 'Merge isolation time', 'Devs 1', 'Devs 2',
       'Different devs', 'Same devs', 'Devs intersection', 'Commits 1',
       'Commits 2', 'Changed files 1', 'Changed files 2',
       'Changed 

In [9]:
chunks_per_project = selected_dataset_2.groupby(['project']).count()['chunk_id']
chunks_per_project

project
CCI-MIT/XCoLab                           6889
CloudStack-extras/CloudStack-archive     1779
Ramblurr/Anki-Android                    1115
SINTEF-9012/ThingML                      1277
TeamDev-Ltd/OpenFaces                    3723
Unidata/thredds                          1442
alexo/wro4j                              2078
alkacon/opencms-core                     1153
android/platform_frameworks_base         4446
apache/accumulo                          5141
apache/directory-server                  1056
apache/lucene-solr                       1570
atlasapi/atlas                           1383
eclipse/jetty.project                    1921
elastic/elasticsearch                    1557
eucalyptus/eucalyptus                    1275
freenet/fred                             1584
getrailo/railo                           1018
hibernate/hibernate-orm                  1249
jgralab/jgralab                          2590
revolsys/com.revolsys.open               1238
sebastianbenz/Jnario      

In [10]:
join_api_selected = pd.merge(api_data, chunks_per_project, on='project', how='inner')
join_api_selected[['project_actual_ownername','chunks', 'chunk_id']]

,project_actual_ownername,chunks,chunk_id
0,sebastianbenz/Jnario,10372,10372
1,CCI-MIT/XCoLab,6889,6889
2,apache/accumulo,5141,5141
3,aosp-mirror/platform_frameworks_base,4446,4446
4,TeamDev-Archive/OpenFaces,3723,3723
5,jgralab/jgralab,2590,2590
6,alexo/wro4j,2078,2078
7,eclipse/jetty.project,1921,1921
8,CloudStack-extras/CloudStack-archive,1779,1779
9,freenet/fred,1584,1584


## Characteristics of the Projects

In [11]:
selected_projects = api_data[api_data.chunks >= 1000]
selected_projects

,id,project,project_new_ownername,chunks,repo_not_found,project_actual_ownername,resourcePath,stargazerCount,isFork,deleteBranchOnMerge,...,issues,pullRequests,releases,watchers,forks,forkCount,commits,contributors,branches,tags
0,3518171,sebastianbenz/Jnario,NaN,10372,False,sebastianbenz/Jnario,/sebastianbenz/Jnario,133.0,False,False,...,146.0,35.0,0.0,19.0,32.0,37.0,1303.0,9.0,19.0,20.0
1,3661343,CCI-MIT/XCoLab,NaN,6889,False,CCI-MIT/XCoLab,/CCI-MIT/XCoLab,15.0,False,False,...,9.0,288.0,0.0,8.0,10.0,10.0,10045.0,25.0,97.0,0.0
2,2524488,apache/accumulo,NaN,5141,False,apache/accumulo,/apache/accumulo,864.0,False,False,...,628.0,1394.0,0.0,88.0,333.0,351.0,10876.0,131.0,3.0,38.0
3,65885,android/platform_frameworks_base,aosp-mirror/platform_frameworks_base,4446,False,aosp-mirror/platform_frameworks_base,/aosp-mirror/platform_frameworks_base,9532.0,False,False,...,0.0,111.0,0.0,911.0,5360.0,5901.0,566598.0,763.0,250.0,1034.0
4,1006053,TeamDev-Ltd/OpenFaces,TeamDev-Archive/OpenFaces,3723,False,TeamDev-Archive/OpenFaces,/TeamDev-Archive/OpenFaces,28.0,False,False,...,173.0,3.0,0.0,19.0,14.0,15.0,3293.0,18.0,8.0,8.0
5,4212733,RealVNC/android-frameworks-base-with-screensho...,VNCAutomotive/android-frameworks-base-with-scr...,2836,False,VNCAutomotive/android-frameworks-base-with-scr...,/VNCAutomotive/android-frameworks-base-with-sc...,4.0,False,False,...,0.0,0.0,0.0,18.0,3.0,3.0,62550.0,83.0,11.0,1.0
6,3405664,jgralab/jgralab,NaN,2590,False,jgralab/jgralab,/jgralab/jgralab,15.0,False,False,...,8.0,0.0,0.0,13.0,3.0,3.0,7630.0,11.0,8.0,70.0
7,2902099,AOKP/frameworks_base_disabled,NaN,2498,False,AOKP/frameworks_base_disabled,/AOKP/frameworks_base_disabled,98.0,False,False,...,1.0,77.0,0.0,8.0,69.0,75.0,78600.0,210.0,3.0,1.0
8,3518362,adetaylor/android-frameworks-base-with-remote-...,NaN,2240,False,adetaylor/android-frameworks-base-with-remote-...,/adetaylor/android-frameworks-base-with-remote...,2.0,False,False,...,0.0,0.0,0.0,1.0,1.0,2.0,62743.0,83.0,3.0,0.0
9,3129899,IceColdSandwich/android_frameworks_base,NaN,2240,False,IceColdSandwich/android_frameworks_base,/IceColdSandwich/android_frameworks_base,13.0,False,False,...,0.0,3.0,0.0,1.0,3.0,3.0,63311.0,204.0,7.0,0.0


In [12]:
join_api_merges_csv = pd.merge(merges_csv, selected_projects, on='project', how='inner')
join_api_merges_csv = join_api_merges_csv.drop(columns=['id_y'])
join_api_merges_csv = join_api_merges_csv.rename(columns={'id_x':'id'})

join_api_merges_csv

,id,project,nr_merges,nr_conflicting_merges,project_new_ownername,chunks,repo_not_found,project_actual_ownername,resourcePath,stargazerCount,...,issues,pullRequests,releases,watchers,forks,forkCount,commits,contributors,branches,tags
0,65885,android/platform_frameworks_base,156732,3355,aosp-mirror/platform_frameworks_base,4446,False,aosp-mirror/platform_frameworks_base,/aosp-mirror/platform_frameworks_base,9532.0,...,0.0,111.0,0.0,911.0,5360.0,5901.0,566598.0,763.0,250.0,1034.0
1,185026,freenet/fred,1076,237,NaN,1584,False,freenet/fred,/freenet/fred,685.0,...,0.0,722.0,48.0,87.0,161.0,171.0,30800.0,71.0,99.0,2015.0
2,206437,apache/directory-server,50,18,NaN,1056,False,apache/directory-server,/apache/directory-server,82.0,...,0.0,41.0,0.0,24.0,60.0,63.0,9797.0,39.0,95.0,51.0
3,217482,Ramblurr/Anki-Android,952,155,NaN,1115,False,Ramblurr/Anki-Android,/Ramblurr/Anki-Android,80.0,...,0.0,0.0,0.0,16.0,33.0,309.0,4888.0,31.0,4.0,0.0
4,223355,alexo/wro4j,1458,395,NaN,2078,False,alexo/wro4j,/alexo/wro4j,444.0,...,125.0,103.0,0.0,35.0,113.0,114.0,3619.0,37.0,55.0,46.0
5,507775,elastic/elasticsearch,2442,235,NaN,1557,False,elastic/elasticsearch,/elastic/elasticsearch,54627.0,...,26815.0,45097.0,44.0,2781.0,19045.0,19739.0,58486.0,3118.0,118.0,293.0
6,726492,getrailo/railo,671,163,NaN,1018,False,getrailo/railo,/getrailo/railo,246.0,...,0.0,441.0,0.0,43.0,86.0,87.0,3990.0,16.0,6.0,164.0
7,762119,atlasapi/atlas,2284,418,NaN,1383,False,atlasapi/atlas,/atlasapi/atlas,32.0,...,26.0,1490.0,0.0,11.0,7.0,9.0,7182.0,45.0,506.0,7.0
8,961036,hibernate/hibernate-orm,152,27,NaN,1249,False,hibernate/hibernate-orm,/hibernate/hibernate-orm,4595.0,...,0.0,3881.0,16.0,321.0,2843.0,2946.0,10772.0,420.0,21.0,225.0
9,1006053,TeamDev-Ltd/OpenFaces,611,178,TeamDev-Archive/OpenFaces,3723,False,TeamDev-Archive/OpenFaces,/TeamDev-Archive/OpenFaces,28.0,...,173.0,3.0,0.0,19.0,14.0,15.0,3293.0,18.0,8.0,8.0


### Projects that are not forks (from the 29 initial projects)

In [13]:
projs = [("freenet/fred", 0),
("apache/directory-server", 0),
("Ramblurr/Anki-Android", 0),
("alexo/wro4j", 0),
("getrailo/railo", 0),
("atlasapi/atlas", 0),
("hibernate/hibernate-orm", 0),
("TeamDev-Ltd/OpenFaces", 0),
("CloudStack-extras/CloudStack-archive", 0),
("zkoss/zk", 0),
("eucalyptus/eucalyptus", 0),
("alkacon/opencms-core", 0),
("SINTEF-9012/ThingML", 0),
("Unidata/thredds", 0),
("apache/accumulo", 0),
("jgralab/jgralab", 0),
("sebastianbenz/Jnario", 0),
("CCI-MIT/XCoLab", 0),
("apache/lucene-solr", 0),
("android/platform_frameworks_base", 0),
("elastic/elasticsearch", 1),
("eclipse/jetty.project", 1),
("revolsys/com.revolsys.open", 1)]

new_projs = pd.DataFrame.from_records(projs, columns=['project', 'NA'])
new_projs

,project,NA
0,freenet/fred,0
1,apache/directory-server,0
2,Ramblurr/Anki-Android,0
3,alexo/wro4j,0
4,getrailo/railo,0
5,atlasapi/atlas,0
6,hibernate/hibernate-orm,0
7,TeamDev-Ltd/OpenFaces,0
8,CloudStack-extras/CloudStack-archive,0
9,zkoss/zk,0


In [14]:
projects23 = pd.merge(join_api_merges_csv, new_projs, on='project', how='inner')
projects23

,id,project,nr_merges,nr_conflicting_merges,project_new_ownername,chunks,repo_not_found,project_actual_ownername,resourcePath,stargazerCount,...,pullRequests,releases,watchers,forks,forkCount,commits,contributors,branches,tags,NA
0,65885,android/platform_frameworks_base,156732,3355,aosp-mirror/platform_frameworks_base,4446,False,aosp-mirror/platform_frameworks_base,/aosp-mirror/platform_frameworks_base,9532.0,...,111.0,0.0,911.0,5360.0,5901.0,566598.0,763.0,250.0,1034.0,0
1,185026,freenet/fred,1076,237,NaN,1584,False,freenet/fred,/freenet/fred,685.0,...,722.0,48.0,87.0,161.0,171.0,30800.0,71.0,99.0,2015.0,0
2,206437,apache/directory-server,50,18,NaN,1056,False,apache/directory-server,/apache/directory-server,82.0,...,41.0,0.0,24.0,60.0,63.0,9797.0,39.0,95.0,51.0,0
3,217482,Ramblurr/Anki-Android,952,155,NaN,1115,False,Ramblurr/Anki-Android,/Ramblurr/Anki-Android,80.0,...,0.0,0.0,16.0,33.0,309.0,4888.0,31.0,4.0,0.0,0
4,223355,alexo/wro4j,1458,395,NaN,2078,False,alexo/wro4j,/alexo/wro4j,444.0,...,103.0,0.0,35.0,113.0,114.0,3619.0,37.0,55.0,46.0,0
5,507775,elastic/elasticsearch,2442,235,NaN,1557,False,elastic/elasticsearch,/elastic/elasticsearch,54627.0,...,45097.0,44.0,2781.0,19045.0,19739.0,58486.0,3118.0,118.0,293.0,1
6,726492,getrailo/railo,671,163,NaN,1018,False,getrailo/railo,/getrailo/railo,246.0,...,441.0,0.0,43.0,86.0,87.0,3990.0,16.0,6.0,164.0,0
7,762119,atlasapi/atlas,2284,418,NaN,1383,False,atlasapi/atlas,/atlasapi/atlas,32.0,...,1490.0,0.0,11.0,7.0,9.0,7182.0,45.0,506.0,7.0,0
8,961036,hibernate/hibernate-orm,152,27,NaN,1249,False,hibernate/hibernate-orm,/hibernate/hibernate-orm,4595.0,...,3881.0,16.0,321.0,2843.0,2946.0,10772.0,420.0,21.0,225.0,0
9,1006053,TeamDev-Ltd/OpenFaces,611,178,TeamDev-Archive/OpenFaces,3723,False,TeamDev-Archive/OpenFaces,/TeamDev-Archive/OpenFaces,28.0,...,3.0,0.0,19.0,14.0,15.0,3293.0,18.0,8.0,8.0,0


In [15]:
selected_projects_table = projects23[["id", "project_actual_ownername", "chunks", "nr_merges", "nr_conflicting_merges", "stargazerCount",
                                             "commits", "contributors", "branches"]]

selected_projects_table = selected_projects_table.rename(columns={"project_actual_ownername": "Project",
                                        "nr_merges": "Merges",
                                        "nr_conflicting_merges": "Conflicting Merges",
                                        "chunks": "Chunks",
                                        "stargazerCount": "Stars",
                                        "commits": "Commits",
                                        "contributors": "Developers",
                                        "branches": "Branches"})
# float columns to int
selected_projects_table[["Chunks", "Stars", "Commits", "Developers", "Branches"]] = \
  selected_projects_table[["Chunks", "Stars", "Commits", "Developers", "Branches"]].astype(int)

selected_projects_table = selected_projects_table.sort_values(by=['Chunks'], ascending=False)
selected_projects_table

,id,Project,Chunks,Merges,Conflicting Merges,Stars,Commits,Developers,Branches
20,3518171,sebastianbenz/Jnario,10372,32,5,133,1303,9,19
21,3661343,CCI-MIT/XCoLab,6889,579,124,15,10045,25,97
17,2524488,apache/accumulo,5141,2548,784,864,10876,131,3
0,65885,aosp-mirror/platform_frameworks_base,4446,156732,3355,9532,566598,763,250
9,1006053,TeamDev-Archive/OpenFaces,3723,611,178,28,3293,18,8
19,3405664,jgralab/jgralab,2590,407,101,15,7630,11,8
4,223355,alexo/wro4j,2078,1458,395,444,3619,37,55
15,2045207,eclipse/jetty.project,1921,1882,524,3093,22856,309,45
10,1022930,CloudStack-extras/CloudStack-archive,1779,829,132,256,12035,39,195
1,185026,freenet/fred,1584,1076,237,685,30800,71,99


### Fixing columns "Commits" and "Developers" to show data from Ghiotto's database

In [16]:
statistics_bd_project_selected = pd.read_csv("../../data/statistics_bd_project_selected.csv")

# fixing project with wrong value
statistics_bd_project_selected.at[0, 'commits'] = 246462

statistics_bd_project_selected

,id,project,commits,merges,conflicting_merges,developers,stars
0,65885,android/platform_frameworks_base,246462,156764,3355,115,3286
1,185026,freenet/fred,30970,1076,237,62,253
2,206437,apache/directory-server,12914,52,18,12,21
3,217482,Ramblurr/Anki-Android,4905,952,155,62,64
4,223355,alexo/wro4j,4565,1458,395,37,318
5,507775,elastic/elasticsearch,35163,2442,235,416,6359
6,726492,getrailo/railo,5215,671,163,15,266
7,762119,atlasapi/atlas,9603,2284,418,28,29
8,961036,hibernate/hibernate-orm,10754,152,27,180,1765
9,1006053,TeamDev-Ltd/OpenFaces,3611,611,178,16,9


In [17]:
commits_devs_from_BD = statistics_bd_project_selected[["id", "commits", "developers", "stars"]]
commits_devs_from_BD

,id,commits,developers,stars
0,65885,246462,115,3286
1,185026,30970,62,253
2,206437,12914,12,21
3,217482,4905,62,64
4,223355,4565,37,318
5,507775,35163,416,6359
6,726492,5215,15,266
7,762119,9603,28,29
8,961036,10754,180,1765
9,1006053,3611,16,9


In [18]:
table_projects = pd.merge(selected_projects_table, 
                                   commits_devs_from_BD,
                                   on='id', how='inner')\
        .drop(columns=['id', 'Branches', 'Developers', 'Commits', 'Stars'])\
        .rename(columns={"developers": "Developers",
                         "commits": "Commits",
                         "stars": "Stars"})

# fixing project with value 0
# table_projects.at[3, 'Commits'] = 246462

# table_projects = table_projects.append(table_projects.median(numeric_only=True), ignore_index=True)
# table_projects.at[29, 'Project'] = 'Median'
table_projects

,Project,Chunks,Merges,Conflicting Merges,Commits,Developers,Stars
0,sebastianbenz/Jnario,10372,32,5,1635,8,53
1,CCI-MIT/XCoLab,6889,579,124,3865,12,4
2,apache/accumulo,5141,2548,784,8364,56,187
3,aosp-mirror/platform_frameworks_base,4446,156732,3355,246462,115,3286
4,TeamDev-Archive/OpenFaces,3723,611,178,3611,16,9
5,jgralab/jgralab,2590,407,101,7871,10,13
6,alexo/wro4j,2078,1458,395,4565,37,318
7,eclipse/jetty.project,1921,1882,524,12935,35,602
8,CloudStack-extras/CloudStack-archive,1779,829,132,33294,29,5
9,freenet/fred,1584,1076,237,30970,62,253


In [19]:
table_projects.describe()

,Chunks,Merges,Conflicting Merges,Commits,Developers,Stars
count,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,2487.565217,7941.347826,363.043478,24348.043478,56.434783,645.608696
std,2285.919903,32461.145673,689.125564,49857.912708,87.654189,1449.774896
min,1018.000000,32.000000,4.000000,1635.000000,1.000000,2.000000
25%,1262.000000,346.000000,41.000000,4735.000000,14.500000,18.500000
50%,1557.000000,829.000000,163.000000,10156.000000,33.000000,169.000000
75%,2334.000000,1968.000000,406.500000,24418.000000,51.500000,323.500000
max,10372.000000,156732.000000,3355.000000,246462.000000,416.000000,6359.000000


In [20]:
# pd.merge(table_projects, new_projs, on='Project', how='outer')

In [21]:
# metrics = ['sum', 'median', 'mean', 'std', 'min', quantile_25, quantile_50, quantile_75, 'max']

projects_stats = table_projects.agg({'Chunks' : 'sum',
                    'Merges' : 'sum',
                    'Conflicting Merges' : 'sum',
                    'Commits' : 'sum',
                    'Developers' : 'sum',
                    'Stars' : 'sum'
                   })

projects_stats['Project'] = 'Total'

projects_stats.to_frame().T

,Chunks,Merges,Conflicting Merges,Commits,Developers,Stars,Project
0,57214,182651,8350,560005,1298,14849,Total


In [22]:
# table_with_stats = pd.concat([table_projects, projects_stats])
table_with_stats = table_projects.append(table_projects.sum(numeric_only=True).rename('Total'))

table_with_stats.at['Total', 'Project'] = 'Total'

table_with_stats

,Project,Chunks,Merges,Conflicting Merges,Commits,Developers,Stars
0,sebastianbenz/Jnario,10372.0,32.0,5.0,1635.0,8.0,53.0
1,CCI-MIT/XCoLab,6889.0,579.0,124.0,3865.0,12.0,4.0
2,apache/accumulo,5141.0,2548.0,784.0,8364.0,56.0,187.0
3,aosp-mirror/platform_frameworks_base,4446.0,156732.0,3355.0,246462.0,115.0,3286.0
4,TeamDev-Archive/OpenFaces,3723.0,611.0,178.0,3611.0,16.0,9.0
5,jgralab/jgralab,2590.0,407.0,101.0,7871.0,10.0,13.0
6,alexo/wro4j,2078.0,1458.0,395.0,4565.0,37.0,318.0
7,eclipse/jetty.project,1921.0,1882.0,524.0,12935.0,35.0,602.0
8,CloudStack-extras/CloudStack-archive,1779.0,829.0,132.0,33294.0,29.0,5.0
9,freenet/fred,1584.0,1076.0,237.0,30970.0,62.0,253.0


In [23]:
boxplot = table_projects.boxplot(rot=45, grid=False, column=['Chunks', 'Merges', 'Conflicting Merges', 'Commits', 'Developers', 'Stars'])

### LaTeX output

In [24]:
# with pd.option_context("max_colwidth", None):
#   print(table_projects.to_latex(index=False))
table_with_stats.to_csv('../../data/selected_projects_statistics.csv', index=None)
with pd.option_context("max_colwidth", None):
  print(table_with_stats.to_latex(index=False, float_format="{:,.0f}".format))

\begin{tabular}{lrrrrrr}
\toprule
                             Project &  Chunks &  Merges &  Conflicting Merges &  Commits &  Developers &  Stars \\
\midrule
                sebastianbenz/Jnario &  10,372 &      32 &                   5 &    1,635 &           8 &     53 \\
                      CCI-MIT/XCoLab &   6,889 &     579 &                 124 &    3,865 &          12 &      4 \\
                     apache/accumulo &   5,141 &   2,548 &                 784 &    8,364 &          56 &    187 \\
aosp-mirror/platform\_frameworks\_base &   4,446 & 156,732 &               3,355 &  246,462 &         115 &  3,286 \\
           TeamDev-Archive/OpenFaces &   3,723 &     611 &                 178 &    3,611 &          16 &      9 \\
                     jgralab/jgralab &   2,590 &     407 &                 101 &    7,871 &          10 &     13 \\
                         alexo/wro4j &   2,078 &   1,458 &                 395 &    4,565 &          37 &    318 \\
               eclipse/jett

## Attributes

In [25]:
attributes = pd.read_csv("../../data/attributes.csv")
attributes

,Category,Attribute,Type,Scope,Name in the dataset,References,Reference source,Details,Reasoning,Easy to retrieve?,How to retrieve,Status
0,Authorship,Common developers,Numeric / Discrete,Merge,Same devs,"Menezes et al., 2020",Related work,Number of authors that contributed to both bra...,Our hypothesis is that conflicts that are clas...,Yes,Use the author's tool https://github.com/catar...,Collected
1,Authorship,Different developers,Numeric / Discrete,Merge,Different devs,"Menezes et al., 2020",Related work,Number of different authors that participated ...,Attributes that are related to the complexity ...,Yes,Use the author's tool https://github.com/catar...,Collected
2,Authorship,Percentage of self-conflicts,Numeric / Continuous,Chunk,self_conflict_perc,-,Authors' intuition,Percentage of authors that contributed to both...,Our hypothesis is that conflicts that are clas...,Yes,Use our own scripts.,Collected
3,Authorship,Author contribution,Binary,Chunk,One column per author project,-,Authors' intuition,Have the author contributed to any line of the...,Our hypothesis is that the choice of specific ...,Yes,Use our own scripts.,Collected
4,Authorship,Developers intersection,Ordinal,Merge,Devs intersection,"Menezes et al., 2020",Related work,Degree of intersection of authors between the ...,"According to Menezes et al., 2020, the so call...",Yes,Use the author's tool https://github.com/catar...,Collected
5,Authorship,Multiple authors,Binary,Merge,has_multiple_devs_on_each_side,"Catarina's papers (FSE 2016, TSE 2019)",-,Does the merge have contributions from multipl...,Our hypothesis is that explicit merge (branch ...,Yes,Derive from number of commit authors attribute,NaN
6,Complexity,Chunk cyclomatic complexity,Numeric / Discrete,Chunk,"leftCC, rightCC",-,Authors' intuition,Cyclomatic Complexity of a version (left or ri...,Attributes that are related to the complexity ...,Yes,Use our own scripts.,Collected
7,Complexity,File cyclomatic complexity,Numeric / Discrete,File,fileCC,-,Authors' intuition,Cyclomatic Complexity of the chunk's file.,Attributes that are related to the complexity ...,Yes,Use our own scripts.,Collected
8,Composition,Constructs,Binary,Chunk,"kind_conflict (unprocessed), one column for ea...",-,Authors' intuition,Programming language constructs (e.g. attribut...,"According to Ghiotto et al., 2018, some combin...",Yes,This attribute is already present in the confl...,Collected
9,Location,Chunk position,Numeric / Discrete,Chunk,chunkPosition,-,Authors' intuition,If we break down the file where the chunk happ...,Our hypothesis is that conflicts that happen i...,Yes,Use our own scripts.,Collected


In [26]:
attributes_used = attributes[attributes["Name in the dataset"].notna()]
print(f"{attributes_used.shape[0]} atributes used at the dataset.") 

25 atributes used at the dataset.


In [27]:
attributes_used

,Category,Attribute,Type,Scope,Name in the dataset,References,Reference source,Details,Reasoning,Easy to retrieve?,How to retrieve,Status
0,Authorship,Common developers,Numeric / Discrete,Merge,Same devs,"Menezes et al., 2020",Related work,Number of authors that contributed to both bra...,Our hypothesis is that conflicts that are clas...,Yes,Use the author's tool https://github.com/catar...,Collected
1,Authorship,Different developers,Numeric / Discrete,Merge,Different devs,"Menezes et al., 2020",Related work,Number of different authors that participated ...,Attributes that are related to the complexity ...,Yes,Use the author's tool https://github.com/catar...,Collected
2,Authorship,Percentage of self-conflicts,Numeric / Continuous,Chunk,self_conflict_perc,-,Authors' intuition,Percentage of authors that contributed to both...,Our hypothesis is that conflicts that are clas...,Yes,Use our own scripts.,Collected
3,Authorship,Author contribution,Binary,Chunk,One column per author project,-,Authors' intuition,Have the author contributed to any line of the...,Our hypothesis is that the choice of specific ...,Yes,Use our own scripts.,Collected
4,Authorship,Developers intersection,Ordinal,Merge,Devs intersection,"Menezes et al., 2020",Related work,Degree of intersection of authors between the ...,"According to Menezes et al., 2020, the so call...",Yes,Use the author's tool https://github.com/catar...,Collected
5,Authorship,Multiple authors,Binary,Merge,has_multiple_devs_on_each_side,"Catarina's papers (FSE 2016, TSE 2019)",-,Does the merge have contributions from multipl...,Our hypothesis is that explicit merge (branch ...,Yes,Derive from number of commit authors attribute,NaN
6,Complexity,Chunk cyclomatic complexity,Numeric / Discrete,Chunk,"leftCC, rightCC",-,Authors' intuition,Cyclomatic Complexity of a version (left or ri...,Attributes that are related to the complexity ...,Yes,Use our own scripts.,Collected
7,Complexity,File cyclomatic complexity,Numeric / Discrete,File,fileCC,-,Authors' intuition,Cyclomatic Complexity of the chunk's file.,Attributes that are related to the complexity ...,Yes,Use our own scripts.,Collected
8,Composition,Constructs,Binary,Chunk,"kind_conflict (unprocessed), one column for ea...",-,Authors' intuition,Programming language constructs (e.g. attribut...,"According to Ghiotto et al., 2018, some combin...",Yes,This attribute is already present in the confl...,Collected
9,Location,Chunk position,Numeric / Discrete,Chunk,chunkPosition,-,Authors' intuition,If we break down the file where the chunk happ...,Our hypothesis is that conflicts that happen i...,Yes,Use our own scripts.,Collected


In [28]:
attrs_table = attributes_used[['Category', 'Attribute', 'Type', 'Scope', 'Details']]\
                .sort_values(by=['Category', 'Scope'])\
                .rename(columns={'Details':'Description'})
attrs_table

,Category,Attribute,Type,Scope,Description
2,Authorship,Percentage of self-conflicts,Numeric / Continuous,Chunk,Percentage of authors that contributed to both...
3,Authorship,Author contribution,Binary,Chunk,Have the author contributed to any line of the...
0,Authorship,Common developers,Numeric / Discrete,Merge,Number of authors that contributed to both bra...
1,Authorship,Different developers,Numeric / Discrete,Merge,Number of different authors that participated ...
4,Authorship,Developers intersection,Ordinal,Merge,Degree of intersection of authors between the ...
5,Authorship,Multiple authors,Binary,Merge,Does the merge have contributions from multipl...
6,Complexity,Chunk cyclomatic complexity,Numeric / Discrete,Chunk,Cyclomatic Complexity of a version (left or ri...
7,Complexity,File cyclomatic complexity,Numeric / Discrete,File,Cyclomatic Complexity of the chunk's file.
8,Composition,Constructs,Binary,Chunk,Programming language constructs (e.g. attribut...
9,Location,Chunk position,Numeric / Discrete,Chunk,If we break down the file where the chunk happ...


In [29]:
with pd.option_context("max_colwidth", None):
    txt = attrs_table.to_latex(index=False,
                               float_format="{:,.0f}".format,
                               column_format="p{0.08\linewidth}p{0.2\linewidth}\
                               p{0.15\linewidth}p{0.04\linewidth}p{0.4\linewidth}")
    x = re.sub("[ \t]+", " ", txt)
    print(x)

\begin{tabular}{p{0.08\linewidth}p{0.2\linewidth} p{0.15\linewidth}p{0.04\linewidth}p{0.4\linewidth}}
\toprule
 Category & Attribute & Type & Scope & Description \\
\midrule
 Authorship & Percentage of self-conflicts & Numeric / Continuous & Chunk & Percentage of authors that contributed to both versions, among all authors who contributed to the chunk. \\
 Authorship & Author contribution & Binary & Chunk & Have the author contributed to any line of the chunk? \\
 Authorship & Common developers & Numeric / Discrete & Merge & Number of authors that contributed to both branches of the merge \\
 Authorship & Different developers & Numeric / Discrete & Merge & Number of different authors that participated in the merge, including all commits between the common ancestor and the merge commit. \\
 Authorship & Developers intersection & Ordinal & Merge & Degree of intersection of authors between the merged branches: no intersection, some intersection, or all developers in common. \\
 Authorship

# Experiment results summary

In [30]:
summary = pd.read_csv('../../data/results/experiment_results.csv')
'''
    Given two dataframes, reorder the second based on the first
'''
def reorder_projects(original_df, second_df):
    projects_order = list(original_df['Project'])
    projects_order.remove('Total')
    new_df_data = []
    for project in projects_order:
        rows = second_df[second_df['project'] == project]
        if len(rows) == 1:
            new_df_data.append(rows.iloc[0])
    new_df_data.append(second_df[second_df['project'] == 'Overall'].iloc[0])
    return pd.DataFrame(new_df_data, columns=second_df.columns)

def get_new_project_name(project_name):
    project_row = api_data[api_data['project'] == project_name]
    if len(project_row) == 1:
        if not pd.isnull(project_row.iloc[0]['project_new_ownername']):
            return project_row.iloc[0]['project_new_ownername'] 
    return project_name

summary['project'] = summary['project'].apply(get_new_project_name)
summary = reorder_projects(table_with_stats, summary).reset_index(drop=True)

In [31]:
summary

,project,accuracy_cv,baseline,accuracy_test,norm._improv.,precision,recall,f1-score
0,sebastianbenz/Jnario,0.999,1.000,1.000,0.000,0.999,1.000,0.999
1,CCI-MIT/XCoLab,0.976,0.573,0.980,0.953,0.978,0.980,0.978
2,apache/accumulo,0.863,0.642,0.870,0.637,0.863,0.870,0.864
3,aosp-mirror/platform_frameworks_base,0.817,0.660,0.823,0.479,0.812,0.823,0.815
4,TeamDev-Archive/OpenFaces,0.969,0.942,0.968,0.448,0.961,0.968,0.964
5,jgralab/jgralab,0.866,0.504,0.882,0.762,0.877,0.882,0.877
6,alexo/wro4j,0.584,0.325,0.617,0.433,0.608,0.617,0.609
7,CloudStack-extras/CloudStack-archive,0.806,0.428,0.814,0.675,0.830,0.814,0.817
8,freenet/fred,0.679,0.431,0.684,0.445,0.667,0.684,0.671
9,apache/lucene-solr,0.646,0.267,0.623,0.486,0.613,0.623,0.609


In [32]:
print(summary.to_latex(index=False))

\begin{tabular}{lrrrrrrr}
\toprule
                             project &  accuracy\_cv &  baseline &  accuracy\_test &  norm.\_improv. &  precision &  recall &  f1-score \\
\midrule
                sebastianbenz/Jnario &        0.999 &     1.000 &          1.000 &          0.000 &      0.999 &   1.000 &     0.999 \\
                      CCI-MIT/XCoLab &        0.976 &     0.573 &          0.980 &          0.953 &      0.978 &   0.980 &     0.978 \\
                     apache/accumulo &        0.863 &     0.642 &          0.870 &          0.637 &      0.863 &   0.870 &     0.864 \\
aosp-mirror/platform\_frameworks\_base &        0.817 &     0.660 &          0.823 &          0.479 &      0.812 &   0.823 &     0.815 \\
           TeamDev-Archive/OpenFaces &        0.969 &     0.942 &          0.968 &          0.448 &      0.961 &   0.968 &     0.964 \\
                     jgralab/jgralab &        0.866 &     0.504 &          0.882 &          0.762 &      0.877 &   0.882 &     0.877 \\

# Experiments results by class

In [33]:
import re
by_class = pd.read_csv('../../data/results/experiment_by_class.csv')
by_class = by_class.round(3)
def fix_model_name(model_name):
    if 'baseline_' in model_name:
        return re.sub('baseline_*.*', 'Baseline', model_name)
#         return model_name.replace('baseline_*', 'Baseline')
    if 'random forest' in model_name:
        return model_name.replace('random forest', 'Random Forest')
by_class['model'] = by_class['model'].apply(fix_model_name)
by_class

,target_class,model,precision,recall,f1-score
0,Combination,Baseline,0.101,1.000,0.171
1,Combination,Random Forest,0.581,0.557,0.552
2,Version 1,Baseline,0.519,1.000,0.649
3,Version 1,Random Forest,0.846,0.868,0.856
4,Version 2,Baseline,0.237,1.000,0.352
5,Version 2,Random Forest,0.746,0.739,0.740
6,ConcatenationV1V2,Baseline,0.041,1.000,0.075
7,ConcatenationV1V2,Random Forest,0.786,0.510,0.594
8,ConcatenationV2V1,Baseline,0.008,1.000,0.017
9,ConcatenationV2V1,Random Forest,0.767,0.502,0.542


In [34]:
print(by_class.to_latex(index=False))

\begin{tabular}{llrrr}
\toprule
     target\_class &         model &  precision &  recall &  f1-score \\
\midrule
      Combination &      Baseline &      0.101 &   1.000 &     0.171 \\
      Combination & Random Forest &      0.581 &   0.557 &     0.552 \\
        Version 1 &      Baseline &      0.519 &   1.000 &     0.649 \\
        Version 1 & Random Forest &      0.846 &   0.868 &     0.856 \\
        Version 2 &      Baseline &      0.237 &   1.000 &     0.352 \\
        Version 2 & Random Forest &      0.746 &   0.739 &     0.740 \\
ConcatenationV1V2 &      Baseline &      0.041 &   1.000 &     0.075 \\
ConcatenationV1V2 & Random Forest &      0.786 &   0.510 &     0.594 \\
ConcatenationV2V1 &      Baseline &      0.008 &   1.000 &     0.017 \\
ConcatenationV2V1 & Random Forest &      0.767 &   0.502 &     0.542 \\
           Manual &      Baseline &      0.119 &   1.000 &     0.205 \\
           Manual & Random Forest &      0.622 &   0.561 &     0.582 \\
             None &   

# Attributes importance top-10

In [35]:
attributes_importance = pd.read_csv('../../data/results/attributes_importance.csv').head(10)

In [36]:
attributes_scope = pd.read_csv('../../data/results/attributes_scope.csv')
attributes_importance = attributes_importance.merge(attributes_scope)[['attribute', 'scope', 'average_information_gain', 'average_rank']].round(2)
attributes_importance

,attribute,scope,average_information_gain,average_rank
0,left_lines_added,Merge,1.36,11.05
1,Commits 1,Merge,1.22,14.30
2,left_lines_removed,Merge,1.21,13.70
3,chunkAbsSize,Chunk,1.18,18.75
4,right_lines_removed,Merge,1.14,16.15
5,keyword_remove,Merge,1.06,19.25
6,keyword_use,Merge,1.04,18.75
7,Changed files 1,Merge,1.03,19.10
8,right_lines_added,Merge,1.00,17.25
9,keyword_fix,Merge,0.99,18.05


In [37]:
print(attributes_importance.to_latex(index=False))

\begin{tabular}{llrr}
\toprule
          attribute & scope &  average\_information\_gain &  average\_rank \\
\midrule
   left\_lines\_added & Merge &                      1.36 &         11.05 \\
          Commits 1 & Merge &                      1.22 &         14.30 \\
 left\_lines\_removed & Merge &                      1.21 &         13.70 \\
       chunkAbsSize & Chunk &                      1.18 &         18.75 \\
right\_lines\_removed & Merge &                      1.14 &         16.15 \\
     keyword\_remove & Merge &                      1.06 &         19.25 \\
        keyword\_use & Merge &                      1.04 &         18.75 \\
    Changed files 1 & Merge &                      1.03 &         19.10 \\
  right\_lines\_added & Merge &                      1.00 &         17.25 \\
        keyword\_fix & Merge &                      0.99 &         18.05 \\
\bottomrule
\end{tabular}



# Top-10 Language Constructs information gain

In [38]:
constructs_top10 = pd.read_csv('../../data/results/constructs_ig.csv')
constructs_top10 = constructs_top10[constructs_top10.construct != 'Overall']
constructs_top10 = constructs_top10.sort_values('avg_information_gain', ascending=False).head(10).round(2)
constructs_top10

,construct,avg_information_gain,avg_rank
13,Method invocation,0.75,26.65
17,Variable,0.42,34.35
12,Comment,0.38,33.65
34,If statement,0.33,38.10
4,Attribute,0.31,38.80
37,Import,0.28,40.40
9,Method signature,0.25,39.50
35,Method declaration,0.16,43.90
20,Annotation,0.13,50.70
1,Return statement,0.13,46.15


In [39]:
print(constructs_top10.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
         construct &  avg\_information\_gain &  avg\_rank \\
\midrule
 Method invocation &                  0.75 &     26.65 \\
          Variable &                  0.42 &     34.35 \\
           Comment &                  0.38 &     33.65 \\
      If statement &                  0.33 &     38.10 \\
         Attribute &                  0.31 &     38.80 \\
            Import &                  0.28 &     40.40 \\
  Method signature &                  0.25 &     39.50 \\
Method declaration &                  0.16 &     43.90 \\
        Annotation &                  0.13 &     50.70 \\
  Return statement &                  0.13 &     46.15 \\
\bottomrule
\end{tabular}



# Language construct top-1 per project

In [40]:
constructs_per_project = pd.read_csv('../../data/results/constructs_importance_projects.csv')
def fix_project_name(project_name):
    return project_name.replace('__','/')
constructs_per_project['project'] = constructs_per_project['project'].apply(fix_project_name)
constructs_per_project['project'] = constructs_per_project['project'].apply(get_new_project_name)
constructs_per_project = reorder_projects(table_with_stats, constructs_per_project).reset_index(drop=True)
constructs_per_project['information_gain'] = constructs_per_project['information_gain'].round(3)
constructs_per_project

,project,construct,information_gain,rank
0,sebastianbenz/Jnario,If statement,0.009,1.00
1,CCI-MIT/XCoLab,If statement,0.813,38.00
2,apache/accumulo,Method invocation,0.986,33.00
3,aosp-mirror/platform_frameworks_base,Method invocation,0.820,26.00
4,TeamDev-Archive/OpenFaces,Comment,0.301,24.00
5,jgralab/jgralab,Method invocation,0.696,42.00
6,alexo/wro4j,Method invocation,1.061,13.00
7,CloudStack-extras/CloudStack-archive,Method invocation,0.937,34.00
8,freenet/fred,Method invocation,1.160,27.00
9,apache/lucene-solr,Method invocation,1.398,18.00


In [41]:
print(constructs_per_project.to_latex(index=False))

\begin{tabular}{llrr}
\toprule
                             project &         construct &  information\_gain &  rank \\
\midrule
                sebastianbenz/Jnario &      If statement &             0.009 &  1.00 \\
                      CCI-MIT/XCoLab &      If statement &             0.813 & 38.00 \\
                     apache/accumulo & Method invocation &             0.986 & 33.00 \\
aosp-mirror/platform\_frameworks\_base & Method invocation &             0.820 & 26.00 \\
           TeamDev-Archive/OpenFaces &           Comment &             0.301 & 24.00 \\
                     jgralab/jgralab & Method invocation &             0.696 & 42.00 \\
                         alexo/wro4j & Method invocation &             1.061 & 13.00 \\
CloudStack-extras/CloudStack-archive & Method invocation &             0.937 & 34.00 \\
                        freenet/fred & Method invocation &             1.160 & 27.00 \\
                  apache/lucene-solr & Method invocation &             1.398 

# Developers importance per project

In [42]:
developers_per_project = pd.read_csv('../../data/results/developers_importance.csv')
developers_per_project['project'] = developers_per_project['project'].apply(fix_project_name)
developers_per_project['project'] = developers_per_project['project'].apply(get_new_project_name)
developers_per_project = reorder_projects(table_with_stats, developers_per_project).reset_index(drop=True)
developers_per_project['information_gain'] = developers_per_project['information_gain'].round(3)
developers_per_project = developers_per_project.drop('author', axis=1)
developers_per_project

,project,information_gain,rank
0,sebastianbenz/Jnario,0.000,13.0
1,CCI-MIT/XCoLab,0.908,35.0
2,apache/accumulo,1.063,31.0
3,aosp-mirror/platform_frameworks_base,0.289,34.0
4,TeamDev-Archive/OpenFaces,0.422,13.0
5,jgralab/jgralab,0.977,36.0
6,alexo/wro4j,2.092,1.0
7,CloudStack-extras/CloudStack-archive,0.921,35.0
8,freenet/fred,2.007,2.0
9,apache/lucene-solr,1.336,19.0


In [43]:
print(developers_per_project.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
                             project &  information\_gain &  rank \\
\midrule
                sebastianbenz/Jnario &             0.000 &  13.0 \\
                      CCI-MIT/XCoLab &             0.908 &  35.0 \\
                     apache/accumulo &             1.063 &  31.0 \\
aosp-mirror/platform\_frameworks\_base &             0.289 &  34.0 \\
           TeamDev-Archive/OpenFaces &             0.422 &  13.0 \\
                     jgralab/jgralab &             0.977 &  36.0 \\
                         alexo/wro4j &             2.092 &   1.0 \\
CloudStack-extras/CloudStack-archive &             0.921 &  35.0 \\
                        freenet/fred &             2.007 &   2.0 \\
                  apache/lucene-solr &             1.336 &  19.0 \\
                     Unidata/thredds &             0.851 &   8.0 \\
                      atlasapi/atlas &             1.099 &   8.0 \\
                            zkoss/zk &             1.485 &  11.0 \\
      